In [1]:
import requests

In [2]:
url = "https://www.myntra.com/beauty-page-skincare-bath-body?sort=popularity" 
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")
print(soup.prettify()[:500])

<!DOCTYPE html>
<html lang="en">
 <head>
  <title>
   Beauty Page Skincare Bath Body - Buy Beauty Page Skincare Bath Body online in India
  </title>
  <meta content="#fffff" name="theme-color"/>
  <link href="https://constant.myntassets.com" rel="dns-prefetch preconnect"/>
  <link href="https://assets.myntassets.com" rel="dns-prefetch preconnect"/>
  <link href="https//www.googletagmanager.com" rel="dns-prefetch preconnect"/>
  <link href="https//www.google-analytics.com" rel="dns-prefetch preco


In [35]:
#products = []
#product_names = {}

def scrape_nykaa_products(url_template, skin_type, max_products=500):
    page = 13
    numprods = 0
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}
    
    while numprods < max_products:
        current_url = url_template.format(page=page)
        print(f"Fetching: {current_url}")  # Debugging: Print the current URL
        
        response = requests.get(current_url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}, stopping.")
            break

        soup = BeautifulSoup(response.text, "html.parser")
        product_cards = soup.find_all("div", class_="productWrapper css-17nge1h")

        if not product_cards:
            print("No more products found, stopping.")
            break  # Stop if no products are found on the page

        for product in product_cards:
            numprods += 1
            if numprods >= max_products:
                print("Reached max product limit, stopping.")
                return products

            # Extract product details
            name_tag = product.find("div", class_="css-xrzmfa")
            price_tag = product.find("span", class_="css-111z9ua")
            link_tag = product.find("a")
            image_tag = product.find("img")  # ### ADDED: Extract image URL ###

            product_name = name_tag.text.strip() if name_tag else "N/A"
            product_price = price_tag.text.strip() if price_tag else "N/A"
            product_url = "https://www.nykaa.com" + link_tag["href"] if link_tag else "N/A"
            product_image = image_tag["src"] if image_tag else "N/A"  # ### ADDED: Image URL ###

            # Check if product already exists
            if product_name in product_names:
                index = product_names[product_name]
                existing_skin_type = products[index]["Skin Type"]

                # Avoid duplicate skin types
                if skin_type not in existing_skin_type.split(", "):
                    products[index]["Skin Type"] += f", {skin_type}"
            else:
                # Add a new product entry
                products.append({
                    "Name": product_name,
                    "Price": product_price,
                    "URL": product_url,
                    "Image": product_image,  # ### ADDED: Store image ###
                    "Skin Type": skin_type
                })
                product_names[product_name] = len(products) - 1
        page=page+1
    return products


In [23]:
normal_skin_url = "https://www.nykaa.com/skin/c/8377?page_no={page}&sort=popularity&eq=desktop&skin_type_filter=10780"
normal_skin_products = scrape_nykaa_products(normal_skin_url, "normal")

Fetching: https://www.nykaa.com/skin/c/8377?page_no=11&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=12&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=13&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=14&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=15&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=16&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=17&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=18&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=19&sort=popularity&eq=desktop&skin_type_filter=10780
Fetching: https://www.nykaa.com/skin/c/8377?page_no=20&

KeyboardInterrupt: 

In [28]:
dry_skin_url = "https://www.nykaa.com/skin/c/8377?page_no={page}&sort=popularity&eq=desktop&skin_type_filter=10781"
dry_skin_products = scrape_nykaa_products(dry_skin_url, "dry")

Fetching: https://www.nykaa.com/skin/c/8377?page_no=11&sort=popularity&eq=desktop&skin_type_filter=10781
Fetching: https://www.nykaa.com/skin/c/8377?page_no=12&sort=popularity&eq=desktop&skin_type_filter=10781
Fetching: https://www.nykaa.com/skin/c/8377?page_no=13&sort=popularity&eq=desktop&skin_type_filter=10781
Fetching: https://www.nykaa.com/skin/c/8377?page_no=14&sort=popularity&eq=desktop&skin_type_filter=10781
Fetching: https://www.nykaa.com/skin/c/8377?page_no=15&sort=popularity&eq=desktop&skin_type_filter=10781
Fetching: https://www.nykaa.com/skin/c/8377?page_no=16&sort=popularity&eq=desktop&skin_type_filter=10781
Fetching: https://www.nykaa.com/skin/c/8377?page_no=17&sort=popularity&eq=desktop&skin_type_filter=10781


ConnectionError: ('Connection aborted.', HTTPException('got more than 100 headers'))

In [16]:
oily_skin_url = "https://www.nykaa.com/skin/c/8377?page_no={page}&sort=popularity&eq=desktop&skin_type_filter=10779"
oily_skin_products = scrape_nykaa_products(oily_skin_url, "oily")

Fetching: https://www.nykaa.com/skin/c/8377?page_no=1&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=2&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=3&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=4&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=5&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=6&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=7&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=8&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=9&sort=popularity&eq=desktop&skin_type_filter=10779
Fetching: https://www.nykaa.com/skin/c/8377?page_no=10&sort=popu

ConnectionError: ('Connection aborted.', HTTPException('got more than 100 headers'))

In [34]:
sens_skin_url = "https://www.nykaa.com/skin/c/8377?page_no={page}&sort=popularity&eq=desktop&skin_type_filter=10778"
sens_skin_products = scrape_nykaa_products(sens_skin_url, "sensitive")

Fetching: https://www.nykaa.com/skin/c/8377?page_no=20&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=21&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=22&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=23&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=24&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=25&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=26&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=27&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=28&sort=popularity&eq=desktop&skin_type_filter=10778
Fetching: https://www.nykaa.com/skin/c/8377?page_no=29&

KeyboardInterrupt: 

In [36]:
comb_skin_url = "https://www.nykaa.com/skin/c/8377?page_no={page}&sort=popularity&eq=desktop&skin_type_filter=10782"
comb_skin_products = scrape_nykaa_products(comb_skin_url, "combination")

Fetching: https://www.nykaa.com/skin/c/8377?page_no=13&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=14&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=15&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=16&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=17&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=18&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=19&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=20&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=21&sort=popularity&eq=desktop&skin_type_filter=10782
Fetching: https://www.nykaa.com/skin/c/8377?page_no=22&

In [37]:
len(products)

1374

In [39]:
products[:5]

[{'Name': 'Neutrogena Hydro Boost Hyaluronic Acid Water Gel Light Face ...',
  'Price': '₹750',
  'URL': 'https://www.nykaa.com/neutrogena-hydro-boost-water-gel/p/875156?productId=875156&skuId=362587&pps=1',
  'Image': 'https://images-static.nykaa.com/media/catalog/product/tr:w-220,h-220,cm-pad_resize/8/0/80d60df4891080614470a_01.jpg',
  'Skin Type': 'normal, dry, oily, sensitive, combination'},
 {'Name': 'CeraVe Moisturizer For Dry Skin with Ceramides, Niacinamide ...',
  'Price': '₹359',
  'URL': 'https://www.nykaa.com/cerave-moisturizing-cream-for-dry-to-very-dry-skin-with-ceramides-hyaluronic-acid/p/13169472?productId=13169472&pps=2',
  'Image': 'https://images-static.nykaa.com/media/catalog/product/tr:w-220,h-220,cm-pad_resize/f/4/f455af8CERAV00000001a_1.jpg',
  'Skin Type': 'normal, dry'},
 {'Name': 'SKIN1004 Madagascar Centella Tone Brightening Capsule Ampoul...',
  'Price': '₹2249',
  'URL': 'https://www.nykaa.com/skin1004-madagascar-centella-tone-brightening-capsule-ampoule/p/

In [40]:
import pandas as pd
df = pd.DataFrame(products)

In [44]:
df.head()

,Name,Price,URL,Image,Skin Type
0,Neutrogena Hydro Boost Hyaluronic Acid Water G...,750,https://www.nykaa.com/neutrogena-hydro-boost-w...,https://images-static.nykaa.com/media/catalog/...,"normal, dry, oily, sensitive, combination"
1,CeraVe Moisturizer For Dry Skin with Ceramides...,359,https://www.nykaa.com/cerave-moisturizing-crea...,https://images-static.nykaa.com/media/catalog/...,"normal, dry"
2,SKIN1004 Madagascar Centella Tone Brightening ...,2249,https://www.nykaa.com/skin1004-madagascar-cent...,https://images-static.nykaa.com/media/catalog/...,"normal, sensitive, combination"
3,O3+ D-Tan Professional Pack For De Tan,1411,https://www.nykaa.com/o3-d-tan-pack-300gm/p/20...,https://images-static.nykaa.com/media/catalog/...,normal
4,PLIX 2% Salicylic Acid Jamun Active Acne Contr...,474,https://www.nykaa.com/plix-jamun-active-acne-c...,https://images-static.nykaa.com/media/catalog/...,"normal, dry, oily, sensitive, combination"


In [43]:
df['Price'] = df['Price'].str.replace('₹', '').str.replace(',', '').astype(int)

In [45]:
skin_types = ['normal', 'dry', 'oily', 'sensitive', 'combination']
for skin in skin_types:
    df[skin] = df['Skin Type'].apply(lambda x: 1 if skin in x else 0)
df.drop(columns=['Skin Type'], inplace=True)


In [46]:
df.head()

,Name,Price,URL,Image,normal,dry,oily,sensitive,combination
0,Neutrogena Hydro Boost Hyaluronic Acid Water G...,750,https://www.nykaa.com/neutrogena-hydro-boost-w...,https://images-static.nykaa.com/media/catalog/...,1,1,1,1,1
1,CeraVe Moisturizer For Dry Skin with Ceramides...,359,https://www.nykaa.com/cerave-moisturizing-crea...,https://images-static.nykaa.com/media/catalog/...,1,1,0,0,0
2,SKIN1004 Madagascar Centella Tone Brightening ...,2249,https://www.nykaa.com/skin1004-madagascar-cent...,https://images-static.nykaa.com/media/catalog/...,1,0,0,1,1
3,O3+ D-Tan Professional Pack For De Tan,1411,https://www.nykaa.com/o3-d-tan-pack-300gm/p/20...,https://images-static.nykaa.com/media/catalog/...,1,0,0,0,0
4,PLIX 2% Salicylic Acid Jamun Active Acne Contr...,474,https://www.nykaa.com/plix-jamun-active-acne-c...,https://images-static.nykaa.com/media/catalog/...,1,1,1,1,1


In [48]:
duplicates = df[df.duplicated(subset=['Name'], keep=False)]
print(duplicates)

Empty DataFrame
Columns: [Name, Price, URL, Image, normal, dry, oily, sensitive, combination]
Index: []


In [49]:
csv_filename = "nykaa_products.csv"
df.to_csv(csv_filename, index=False)